## Usando ChatGPT para "re-entrevistar" um arquivo de áudio

Passo a passo:
1. Gravar entrevista e converter para .mp3  
2. Transcrever usando [Whisper](https://openai.com/research/whisper)  
3. Transformar transcrição em um json com marcações de tempo  
4. Criar código que gera marcação temporal a partir de uma pergunta  

## Transcrevendo o áudio

In [39]:
import os

os.listdir('../static/audio')

['entrevista.mp3']

In [41]:
# Ver a diferença de modelos aqui: https://github.com/openai/whisper
import whisper

model = whisper.load_model("base") # Modelo mais simples
result = model.transcribe("../static/audio/entrevista.mp3")
print(f' O texto é: \n {result["text"]}')

/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 O texto é: 
  É muito difícil prever qual vai ser o impacto da inteligência artificial nos empregos, mas a gente sabe que algumas indústrias vão ser mais impactadas no primeiro momento que outras. Por exemplo, a gente sabe que atendimento de calcênter, atendimento ao consumidor, isso tem uma chance grande de ser automatizado. Vários serviços de direito, por exemplo, de escritório de vocacia, também podem ser automatizados. Então, a gente deve pensar muito mais em termos de quais tarefas podem ser automatizadas e não empregos completamente. Certamente, a gente vê mais tipos de trabalho que podem ser mais automatizados que outros. O tema de quais empresas podem lucrar mais com a inteligência artificial, eu citaria especialmente a OPNAI, que é a empresa que trouxe esse assunto, a tona via chat.pt e outras funcionalidades. Certamente, a Microsoft que investiu na OPNAI, e acho que foi aqui mais rapidamente incluiu as funcionalidades de inteligência artificial nos seus produtos, no Word, no

In [4]:
# Agora usando o modelo "large", que demora mais mas é bem mais preciso.
model = whisper.load_model("large")
result = model.transcribe("entrevista.mp3")
print(f' The text in video: \n {result["text"]}')

100%|█████████████████████████████████████| 2.87G/2.87G [09:15<00:00, 5.55MiB/s]
/opt/homebrew/lib/python3.11/site-packages/whisper/transcribe.py:114: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


 The text in video: 
  É muito difícil prever qual vai ser o impacto da inteligência artificial nos empregos, mas a gente sabe que algumas indústrias vão ser mais impactadas num primeiro momento que outras. Por exemplo, a gente sabe que atendimento de call center, atendimento ao consumidor, isso tem uma chance grande de ser automatizado. Vários serviços de direito, por exemplo, de escritório de advocacia, também podem ser automatizados. Então a gente deve pensar muito mais em termos de quais tarefas podem ser automatizadas e não empregos completamente. Certamente a gente vê mais tipos de trabalho que podem ser mais automatizados que outros. Em termos de quais empresas podem lucrar mais com a inteligência artificial, eu citaria especialmente a OpenAI, que é a empresa que trouxe esse assunto à tona via chat e pt e outras funcionalidades. Certamente a Microsoft que investiu na OpenAI e acho que foi a que mais rapidamente incluiu as funcionalidades de inteligência artificial nos seus produ

In [13]:
# Agora a gente limpa o json para ficar apenas com dados que precisamos
data = []
for x in result['segments']:
    obj = {}
    obj['start'] = x['start']
    obj['end'] = x['end']
    obj['text'] = x['text']
    data.append(obj)

transcricao = {"text": result['text'], "segments": data}

In [28]:
transcricao

{'text': ' É muito difícil prever qual vai ser o impacto da inteligência artificial nos empregos, mas a gente sabe que algumas indústrias vão ser mais impactadas no primeiro momento que outras. Por exemplo, a gente sabe que atendimento de calcênter, atendimento ao consumidor, isso tem uma chance grande de ser automatizado. Vários serviços de direito, por exemplo, de escritório de vocacia, também podem ser automatizados. Então, a gente deve pensar muito mais em termos de quais tarefas podem ser automatizadas e não empregos completamente. Certamente, a gente vê mais tipos de trabalho que podem ser mais automatizados que outros. O tema de quais empresas podem lucrar mais com a inteligência artificial, eu citaria especialmente a OPNAI, que é a empresa que trouxe esse assunto, a tona via chat.pt e outras funcionalidades. Certamente, a Microsoft que investiu na OPNAI, e acho que foi aqui mais rapidamente incluiu as funcionalidades de inteligência artificial nos seus produtos, no Word, no Pow

## Usando a API da OpenAI

In [27]:
from openai import OpenAI

api = '' #Coloque sua API aqui
client = OpenAI(api_key=api)

pergunta = 'Qual o impacto de inteligência artificial no jornalismo?'


response = client.chat.completions.create(
  model="gpt-4-1106-preview",
  response_format={ "type": "json_object" },
  messages=[
    {"role": "system", "content": "Você é um assistente solícito que responde em JSON. Sua resposta deve ser uma das duas opções: ou 'não sei' ou o 'start' e 'end' contidos na transcrição de uma entrevista."},
    {"role": "user", "content": f"Utilize essa transcrição de entrevista para responder uma pergunta: {transcricao}.\nA pergunta é: {pergunta}"}
  ]
)

print(response.choices[0].message.content)

{
  "start": 328.0,
  "end": 374.0
}
